In [1]:
import tensorflow as tf
import json
import numpy as np
from sklearn.model_selection import train_test_split

2023-06-06 12:36:13.168543: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 12:36:13.242054: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-06 12:36:13.243127: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 12:36:14.792232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_path = '../dataset.json'

# Open the file and load its contents
with open(file_path, 'r') as json_file:
    # Read the file content
    file_content = json_file.read()

    # Split the content by newline to get individual JSON objects
    json_objects = file_content.split('\n')

data = []
# Process each JSON object
for json_object in json_objects:
    if json_object.strip() == '':
        continue  # Skip empty lines
    
    # Parse the JSON object
    data.append(json.loads(json_object))

print(len(data))

X = []
Y = []
Y_flat = []

for i in data:
    X.append(np.array(i['vector']))    
    Y.append(np.array(i['matrix']))
    
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)
Y_train_flat =[i.flatten() for i in Y_train]
Y_test_flat =[i.flatten() for i in Y_test]

492


# Handmade NN

In [3]:
# Define the neural network architecture
def build_crazy_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(2048, activation='relu'),
        tf.keras.layers.Dense(4096, activation='relu'),
        tf.keras.layers.Dense(output_dim)
    ])
    return model
def build_model(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(196)  # Output layer with shape (196,)
    ])
    return model

def build_model_recursive(input_dim, output_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Reshape((1, 256)),  # Reshape input to match LSTM requirements
        # tf.keras.layers.LSTM(512, return_sequences=True),
        tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(256, activation='relu')),
        tf.keras.layers.LSTM(512),
        tf.keras.layers.Dense(output_dim)
    ])
    return model

def build_model_convolutional(input_dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(7*7*128, activation='relu'),
        tf.keras.layers.Reshape((7, 7, 128)),
        tf.keras.layers.Conv2DTranspose(32, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2DTranspose(2, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(1, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'),
        tf.keras.layers.Reshape((14, 14, 1))  # Update the output shape to (14, 14, 1)
    ])
    return model

In [4]:
# Define the input and output dimensions
input_dim = 128  # Dimensionality of the input vector
output_dim = 196  # Dimensionality of the output vector

In [5]:
# Build the model
model = build_model_recursive(input_dim, output_dim)
model_ann = build_model(input_dim, output_dim)
model_conv = build_model_convolutional(input_dim)

# Compile the model
model.compile(optimizer='adam', loss='mse')
model_ann.compile(optimizer='adam', loss='mse')
model_conv.compile(optimizer='adam', loss='mse')

2023-06-06 12:36:17.282672: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-06 12:36:17.310091: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-06-06 12:36:18.030700: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [6]:
model_ann.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=1000)

Epoch 1/1000
12/12 [==============================] - 2s 13ms/step - loss: 47995884.0000
Epoch 2/1000
12/12 [==============================] - 0s 12ms/step - loss: 64869.0469
Epoch 3/1000
12/12 [==============================] - 0s 11ms/step - loss: 3451.6235
Epoch 4/1000
12/12 [==============================] - 0s 12ms/step - loss: 3448.1375
Epoch 5/1000
12/12 [==============================] - 0s 12ms/step - loss: 3448.1372
Epoch 6/1000
12/12 [==============================] - 0s 12ms/step - loss: 3448.1375
Epoch 7/1000
12/12 [==============================] - 0s 12ms/step - loss: 3448.1372
Epoch 8/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.1372
Epoch 9/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.1370
Epoch 10/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.1372
Epoch 11/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.1362
Epoch 12/1000
12/12 [==============================] - 0s 

12/12 [==============================] - 0s 11ms/step - loss: 3448.0916
Epoch 97/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0903
Epoch 98/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0898
Epoch 99/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0891
Epoch 100/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0889
Epoch 101/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0867
Epoch 102/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0872
Epoch 103/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0854
Epoch 104/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0854
Epoch 105/1000
12/12 [==============================] - 0s 13ms/step - loss: 3448.0847
Epoch 106/1000
12/12 [==============================] - 0s 11ms/step - loss: 3448.0833
Epoch 107/1000
12/12 [==============================] - 0s 13

12/12 [==============================] - 0s 11ms/step - loss: 3447.9880
Epoch 191/1000
12/12 [==============================] - 0s 14ms/step - loss: 3447.9863
Epoch 192/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9858
Epoch 193/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9844
Epoch 194/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9832
Epoch 195/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9814
Epoch 196/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9805
Epoch 197/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9788
Epoch 198/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9771
Epoch 199/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9756
Epoch 200/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.9744
Epoch 201/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 10ms/step - loss: 3447.8232
Epoch 285/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8206
Epoch 286/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8188
Epoch 287/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8167
Epoch 288/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8149
Epoch 289/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8127
Epoch 290/1000
12/12 [==============================] - 0s 11ms/step - loss: 3447.8105
Epoch 291/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8088
Epoch 292/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8069
Epoch 293/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8052
Epoch 294/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.8025
Epoch 295/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 10ms/step - loss: 3447.6130
Epoch 379/1000
12/12 [==============================] - 0s 12ms/step - loss: 3447.6104
Epoch 380/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.6082
Epoch 381/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.6057
Epoch 382/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.6030
Epoch 383/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.6003
Epoch 384/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.5986
Epoch 385/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.5955
Epoch 386/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.5933
Epoch 387/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.5911
Epoch 388/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.5881
Epoch 389/1000
12/12 [==============================] - 0s 9ms/st

...

12/12 [==============================] - 0s 11ms/step - loss: 3447.1223
Epoch 568/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1196
Epoch 569/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1172
Epoch 570/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1147
Epoch 571/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1121
Epoch 572/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1099
Epoch 573/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1074
Epoch 574/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1050
Epoch 575/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.1023
Epoch 576/1000
12/12 [==============================] - 0s 10ms/step - loss: 3447.1013
Epoch 577/1000
12/12 [==============================] - 0s 9ms/step - loss: 3447.0962
Epoch 578/1000
12/12 [==============================] - 0s 9ms/step

12/12 [==============================] - 0s 9ms/step - loss: 3446.8750
Epoch 663/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.8730
Epoch 664/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.8706
Epoch 665/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.8662
Epoch 666/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8635
Epoch 667/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8621
Epoch 668/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8586
Epoch 669/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8557
Epoch 670/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8540
Epoch 671/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8508
Epoch 672/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.8496
Epoch 673/1000
12/12 [==============================] - 0s 9ms/ste

12/12 [==============================] - 0s 9ms/step - loss: 3446.6277
Epoch 758/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6252
Epoch 759/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6223
Epoch 760/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6199
Epoch 761/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.6179
Epoch 762/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.6147
Epoch 763/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6128
Epoch 764/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6101
Epoch 765/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6077
Epoch 766/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6055
Epoch 767/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.6023
Epoch 768/1000
12/12 [==============================] - 0s 9ms/step

12/12 [==============================] - 0s 9ms/step - loss: 3446.3811
Epoch 853/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3770
Epoch 854/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3748
Epoch 855/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3716
Epoch 856/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3672
Epoch 857/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3647
Epoch 858/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3621
Epoch 859/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3599
Epoch 860/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3577
Epoch 861/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3542
Epoch 862/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.3521
Epoch 863/1000
12/12 [==============================] - 0s 9ms/step -

12/12 [==============================] - 0s 9ms/step - loss: 3446.1387
Epoch 948/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1355
Epoch 949/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1345
Epoch 950/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1296
Epoch 951/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1267
Epoch 952/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1252
Epoch 953/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1213
Epoch 954/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1206
Epoch 955/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1174
Epoch 956/1000
12/12 [==============================] - 0s 9ms/step - loss: 3446.1152
Epoch 957/1000
12/12 [==============================] - 0s 10ms/step - loss: 3446.1130
Epoch 958/1000
12/12 [==============================] - 0s 10ms/step

In [7]:
model_conv.fit(x=np.array(X_train),y=np.array(Y_train), epochs=1000)

Epoch 1/1000
12/12 [==============================] - 1s 15ms/step - loss: 227236.1875
Epoch 2/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1895
Epoch 3/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 4/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1365
Epoch 5/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 6/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 7/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 8/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 9/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 10/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 11/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 12/1000
12/12 [==============================] - 0s 16m

12/12 [==============================] - 0s 15ms/step - loss: 3448.1338
Epoch 97/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 98/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 99/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 100/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 101/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 102/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1348
Epoch 103/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 104/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 105/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1345
Epoch 106/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 107/1000
12/12 [==============================] - 0s 15

12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 191/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 192/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 193/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1353
Epoch 194/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 195/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 196/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 197/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 198/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 199/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 200/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 201/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 285/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 286/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1340
Epoch 287/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 288/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 289/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 290/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 291/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 292/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 293/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 294/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 295/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 379/1000
12/12 [==============================] - 0s 17ms/step - loss: 3448.1348
Epoch 380/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1348
Epoch 381/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 382/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 383/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 384/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 385/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 386/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 387/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 388/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 389/1000
12/12 [==============================] - 0s

...

12/12 [==============================] - 0s 16ms/step - loss: 3448.1340
Epoch 567/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1357
Epoch 568/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1348
Epoch 569/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1355
Epoch 570/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1353
Epoch 571/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 572/1000
12/12 [==============================] - 0s 17ms/step - loss: 3448.1345
Epoch 573/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 574/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1353
Epoch 575/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 576/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1345
Epoch 577/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 661/1000
12/12 [==============================] - 0s 20ms/step - loss: 3448.1348
Epoch 662/1000
12/12 [==============================] - 0s 17ms/step - loss: 3448.1345
Epoch 663/1000
12/12 [==============================] - 0s 17ms/step - loss: 3448.1348
Epoch 664/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1340
Epoch 665/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 666/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 667/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 668/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 669/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 670/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 671/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 755/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 756/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1353
Epoch 757/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1340
Epoch 758/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1348
Epoch 759/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 760/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 761/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1340
Epoch 762/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 763/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 764/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 765/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 849/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 850/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 851/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1340
Epoch 852/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 853/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 854/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1348
Epoch 855/1000
12/12 [==============================] - 0s 16ms/step - loss: 3448.1345
Epoch 856/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 857/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 858/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 859/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 943/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 944/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 945/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 946/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 947/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 948/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1345
Epoch 949/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1355
Epoch 950/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1353
Epoch 951/1000
12/12 [==============================] - 0s 15ms/step - loss: 3448.1348
Epoch 952/1000
12/12 [==============================] - 0s 17ms/step - loss: 3448.1348
Epoch 953/1000
12/12 [==============================] - 0s

In [8]:
model.fit(x=np.array(X_train),y=np.array(Y_train_flat), epochs=1000)

Epoch 1/1000


2023-06-06 12:41:30.430254: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 12:41:30.433116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 12:41:30.434921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

12/12 [==============================] - 3s 14ms/step - loss: 3447.3979
Epoch 2/1000
12/12 [==============================] - 0s 14ms/step - loss: 3447.2751
Epoch 3/1000
12/12 [==============================] - 0s 15ms/step - loss: 3447.1589
Epoch 4/1000
12/12 [==============================] - 0s 16ms/step - loss: 3447.1318
Epoch 5/1000
12/12 [==============================] - 0s 16ms/step - loss: 3447.0798
Epoch 6/1000
12/12 [==============================] - 0s 14ms/step - loss: 3446.8687
Epoch 7/1000
12/12 [==============================] - 0s 14ms/step - loss: 3446.7144
Epoch 8/1000
12/12 [==============================] - 0s 14ms/step - loss: 3446.4631
Epoch 9/1000
12/12 [==============================] - 0s 15ms/step - loss: 3446.2771
Epoch 10/1000
12/12 [==============================] - 0s 15ms/step - loss: 3446.0911
Epoch 11/1000
12/12 [==============================] - 0s 14ms/step - loss: 3445.8032
Epoch 12/1000
12/12 [==============================] - 0s 14ms/step - loss: 

12/12 [==============================] - 0s 16ms/step - loss: 3437.1880
Epoch 97/1000
12/12 [==============================] - 0s 15ms/step - loss: 3437.0583
Epoch 98/1000
12/12 [==============================] - 0s 15ms/step - loss: 3436.9421
Epoch 99/1000
12/12 [==============================] - 0s 15ms/step - loss: 3436.8713
Epoch 100/1000
12/12 [==============================] - 0s 14ms/step - loss: 3436.8088
Epoch 101/1000
12/12 [==============================] - 0s 14ms/step - loss: 3436.6914
Epoch 102/1000
12/12 [==============================] - 0s 15ms/step - loss: 3436.6567
Epoch 103/1000
12/12 [==============================] - 0s 14ms/step - loss: 3436.5203
Epoch 104/1000
12/12 [==============================] - 0s 16ms/step - loss: 3436.4731
Epoch 105/1000
12/12 [==============================] - 0s 15ms/step - loss: 3436.3992
Epoch 106/1000
12/12 [==============================] - 0s 17ms/step - loss: 3436.2966
Epoch 107/1000
12/12 [==============================] - 0s 14

12/12 [==============================] - 0s 14ms/step - loss: 3432.1355
Epoch 191/1000
12/12 [==============================] - 0s 15ms/step - loss: 3432.0903
Epoch 192/1000
12/12 [==============================] - 0s 14ms/step - loss: 3432.0593
Epoch 193/1000
12/12 [==============================] - 0s 14ms/step - loss: 3432.0273
Epoch 194/1000
12/12 [==============================] - 0s 14ms/step - loss: 3431.9819
Epoch 195/1000
12/12 [==============================] - 0s 14ms/step - loss: 3431.9473
Epoch 196/1000
12/12 [==============================] - 0s 14ms/step - loss: 3431.9272
Epoch 197/1000
12/12 [==============================] - 0s 14ms/step - loss: 3431.9028
Epoch 198/1000
12/12 [==============================] - 0s 16ms/step - loss: 3431.8826
Epoch 199/1000
12/12 [==============================] - 0s 14ms/step - loss: 3431.8210
Epoch 200/1000
12/12 [==============================] - 0s 15ms/step - loss: 3431.7812
Epoch 201/1000
12/12 [==============================] - 0s

...

12/12 [==============================] - 0s 14ms/step - loss: 3429.8130
Epoch 567/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8215
Epoch 568/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8162
Epoch 569/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8201
Epoch 570/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8206
Epoch 571/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8147
Epoch 572/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8164
Epoch 573/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8193
Epoch 574/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8167
Epoch 575/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8130
Epoch 576/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8188
Epoch 577/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 14ms/step - loss: 3429.8076
Epoch 661/1000
12/12 [==============================] - 0s 15ms/step - loss: 3429.8052
Epoch 662/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.8069
Epoch 663/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.8005
Epoch 664/1000
12/12 [==============================] - 0s 15ms/step - loss: 3429.8101
Epoch 665/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7991
Epoch 666/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8052
Epoch 667/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8162
Epoch 668/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8008
Epoch 669/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8110
Epoch 670/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.8032
Epoch 671/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 14ms/step - loss: 3429.8022
Epoch 755/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7822
Epoch 756/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7822
Epoch 757/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7805
Epoch 758/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7815
Epoch 759/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7910
Epoch 760/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7874
Epoch 761/1000
12/12 [==============================] - 0s 15ms/step - loss: 3429.7869
Epoch 762/1000
12/12 [==============================] - 0s 15ms/step - loss: 3429.7849
Epoch 763/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7927
Epoch 764/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7800
Epoch 765/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 14ms/step - loss: 3429.7927
Epoch 849/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7817
Epoch 850/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7832
Epoch 851/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7808
Epoch 852/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7961
Epoch 853/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8225
Epoch 854/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7742
Epoch 855/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7725
Epoch 856/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8008
Epoch 857/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7788
Epoch 858/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7791
Epoch 859/1000
12/12 [==============================] - 0s

12/12 [==============================] - 0s 15ms/step - loss: 3429.7649
Epoch 943/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7676
Epoch 944/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7656
Epoch 945/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7710
Epoch 946/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7737
Epoch 947/1000
12/12 [==============================] - 0s 14ms/step - loss: 3429.7666
Epoch 948/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7651
Epoch 949/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7812
Epoch 950/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.8298
Epoch 951/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7659
Epoch 952/1000
12/12 [==============================] - 0s 13ms/step - loss: 3429.7676
Epoch 953/1000
12/12 [==============================] - 0s

In [9]:
Y_predicted = model.predict(np.array(X_test))
Y_ann_predicted = model_ann.predict(np.array(X_test))
Y_conv_predicted = model_conv.predict(np.array(X_test))

4/4 [==============================] - 0s 3ms/step


2023-06-06 12:44:21.174261: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-06 12:44:21.176680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-06 12:44:21.178589: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4/4 [==============================] - 0s 4ms/step


In [10]:
Y_predicted = [np.reshape(i, (14,14)) for i in Y_predicted]
Y_predicted

[array([[ 1.65971778e-02,  4.26492654e-03,  3.33878770e-03,
          1.09819230e-02,  7.41573330e-03,  1.05289947e-02,
          1.10613257e-02,  1.97522566e-02,  1.35143017e-02,
          5.32177743e-03,  1.70886051e-02,  1.64707638e-02,
          2.49839015e-02,  1.05832689e-01],
        [ 4.26483806e-03, -9.08597745e-03,  7.17668235e-03,
          9.95563529e-03,  1.20375007e-02,  1.03292568e-03,
          1.13647962e-02,  1.28930956e-02,  1.31131476e-02,
          1.74231622e-02,  8.66194721e-03,  7.32060522e-03,
          1.58639103e-02,  1.87074974e-01],
        [ 3.33885849e-03,  7.17670284e-03,  1.92741100e-02,
          6.55151438e-03,  5.02350321e-03,  4.57028951e-03,
          2.61757988e-03,  8.21156800e-03,  1.09160747e-02,
          7.62591045e-03,  3.51302885e-03,  2.09748745e-03,
          2.30352748e-02,  3.27882059e-02],
        [ 1.09810838e-02,  9.95563436e-03,  6.55160844e-03,
          1.00965323e-02,  1.04693500e-02,  1.58132520e-02,
          8.41448456e-03,  1

In [11]:
Y_ann_predicted = [np.reshape(i, (14,14)) for i in Y_ann_predicted]
Y_ann_predicted

[array([[1.06548835e-02, 3.15056252e-03, 3.21792113e-03, 5.57623291e-03,
         5.34527842e-03, 6.34414051e-03, 6.77522318e-03, 7.75867421e-03,
         2.88177351e-03, 5.98538760e-03, 9.42719076e-03, 1.04454132e-02,
         9.97314416e-03, 2.89208144e-02],
        [2.99730338e-03, 6.78211451e-03, 3.68479337e-03, 4.04455326e-03,
         3.30719282e-03, 6.00185071e-04, 6.19945396e-03, 5.68965217e-03,
         5.45747252e-03, 4.78924951e-03, 6.11541234e-03, 8.60033929e-03,
         1.10669546e-02, 1.46563053e-01],
        [5.60308155e-03, 4.00728965e-03, 7.93799572e-03, 3.97998653e-03,
         4.62567294e-03, 6.37491047e-03, 2.02817842e-03, 5.28457994e-03,
         1.06914574e-03, 2.08580005e-03, 6.00217050e-03, 6.21420331e-03,
         1.21884206e-02, 4.91254367e-02],
        [2.84375809e-03, 4.74617211e-03, 4.14836034e-03, 8.46389029e-03,
         7.57837947e-03, 5.68399066e-03, 6.67087361e-03, 8.05427600e-03,
         6.16049813e-03, 9.63394810e-03, 1.20418891e-02, 3.22815729e-03

In [12]:
print(Y_conv_predicted)

[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]


 [[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.

# AutoKeras

In [16]:
import autokeras as ak
import numpy as np

In [17]:
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
train_set = tf.data.Dataset.from_tensor_slices((X_train, Y_train_flat))
test_set = tf.data.Dataset.from_tensor_slices((X_test, Y_test_flat))


In [20]:
# reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.RegressionHead()(output_node)
reg = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)
reg.fit(train_set, epochs=100, validation_split=0.15)
predicted_y = reg.predict(test_set)
# X_train = X_train.reshape(-1, 128)
# Y_train = Y_train.reshape(-1, 14,14)
print(reg.evaluate(test_set))


Trial 3 Complete [00h 00m 11s]
val_loss: 3.0583102703094482

Best val_loss So Far: 3.0583102703094482
Total elapsed time: 00h 00m 50s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
2023-06-06 12:49:53.062418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_50' with dtype double and shape [369,196]
	 [[{{node Placeholder/_50}}]]
2023-06-06 12:49:53.063056: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype double and shape [369,196]
	 [[{{node Placeholder/_9}}]]
2023-06-06 12:49:53.164823: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Pla

2023-06-06 12:49:54.414572: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [369,128]
	 [[{{node Placeholder/_0}}]]
2023-06-06 12:49:54.415119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_68' with dtype double and shape [369,128]
	 [[{{node Placeholder/_68}}]]
2023-06-06 12:49:54.513012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and sha

2023-06-06 12:49:55.522849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_50' with dtype double and shape [369,196]
	 [[{{node Placeholder/_50}}]]
2023-06-06 12:49:55.523405: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype double and shape [369,196]
	 [[{{node Placeholder/_20}}]]
2023-06-06 12:49:55.618079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_69' with dtype double and 

Epoch 1/100


...

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets
2023-06-06 12:50:21.041869: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [123,128]
	 [[{{node Placeholder/_0}}]]
2023-06-06 12:50:21.042833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [123,196]
	 [[{{node Placeholder/_1}}]]


4/4 [==============================] - 0s 2ms/step


2023-06-06 12:50:29.201019: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype double and shape [123,196]
	 [[{{node Placeholder/_9}}]]
2023-06-06 12:50:29.201369: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_28' with dtype double and shape [123,196]
	 [[{{node Placeholder/_28}}]]


4/4 [==============================] - 1s 3ms/step - loss: 7.4606 - mean_squared_error: 7.4606
[7.460628032684326, 7.460628032684326]


In [21]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128)]             0         
                                                                 
 multi_category_encoding (Mu  (None, 128)              0         
 ltiCategoryEncoding)                                            
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                             

In [ ]:
X_test = X_test.reshape(-1, 128)
Y_ak_predicted =reg.predict(X_test)

# Autogoal